## Location Selection for Facilities in Massachusetts

This jupyter notebook consists of MCLP algorithm implementation. MCLP is maximal coverage location problem that aims to find locations that maximize demand coverage. The data used for this analysis is curated using US Census. 

In [4]:
from geopy.distance import geodesic
import pulp
import pandas as pd

In [5]:
def mclp(df, p):

    #Step 1 -- preparing longtitude and latitude data
    df['lat'] = df['lat'].astype(float)
    df['lng'] = df['lng'].astype(float)

    demand_points = list(zip(df['lat'], df['lng']))
    coverage_radius_miles = 7
    n = len(demand_points)

    # Step 2 - Coverage matrix (if a location is covered within 7 mile or not)
    coverage_matrix = []

    for i in range(n):
        covered_by_i = []
        for j in range(n):
            dist = geodesic(demand_points[i], demand_points[j]).miles
            covered_by_i.append(1 if dist <= coverage_radius_miles else 0)
        coverage_matrix.append(covered_by_i)
    
    # Step 3: Optimization model
    model = pulp.LpProblem("MCLP_Same_Demand_Candidates", pulp.LpMaximize)

    # Decision variables
    x = [pulp.LpVariable(f'x_{j}', cat='Binary') for j in range(n)]  # facility at j
    y = [pulp.LpVariable(f'y_{i}', cat='Binary') for i in range(n)]  # demand i covered

    # Objective -- number of demand covered
    model += pulp.lpSum(y)

    # Facility limit
    model += pulp.lpSum(x) <= p

    # Coverage constraints
    for i in range(n):
        model += y[i] <= pulp.lpSum(coverage_matrix[j][i] * x[j] for j in range(n))

    model.solve()

    # Selected facility locations
    selected_indices = [i for i in range(n) if x[i].varValue == 1.0]
    selected_locations = [demand_points[i] for i in selected_indices]

    print("Selected facility locations:", selected_locations)

    get_result(selected_indices, coverage_matrix, df, n)


In [2]:
def get_result(selected_indices, coverage_matrix, df, n):
    for i in selected_indices:
        covered_points = []
        total_population_covered = 0
        for j in range(n):
            if coverage_matrix[i][j]:
                covered_points.append(j)
        print(f"Facility at {df.iloc[i]['city']} (lat={df.iloc[i]['lat']}, lon={df.iloc[i]['lng']}) covers:")
        for j in covered_points:
            total_population_covered += df.iloc[j]['population']
            print(f"  - {df.iloc[j]['city']} (lat={df.iloc[j]['lat']}, lon={df.iloc[j]['lng']})")
        print(f"total population covered: {total_population_covered}")
        print()


## Location Analysis

### Suffolk

In [7]:
suffolk = pd.read_csv("suffolk.csv")

mclp(suffolk, p = 2)

Selected facility locations: [(42.34748, -71.08199)]
Facility at Boston (lat=42.34748, lon=-71.08199) covers:
  - Revere (lat=42.41887, lon=-71.004)
  - Dorchester Center (lat=42.28583, lon=-71.07114)
  - East Boston (lat=42.37324, lon=-71.01616)
  - Brighton (lat=42.34989, lon=-71.15417)
  - Jamaica Plain (lat=42.30823, lon=-71.11515)
  - Chelsea (lat=42.39587, lon=-71.03251)
  - Hyde Park (lat=42.25505, lon=-71.12662)
  - South Boston (lat=42.33455, lon=-71.03875)
  - Dorchester (lat=42.31578, lon=-71.0562)
  - Roslindale (lat=42.28392, lon=-71.12468)
  - Boston (lat=42.34046, lon=-71.09481)
  - Boston (lat=42.33806, lon=-71.07057)
  - Dorchester (lat=42.3057, lon=-71.08693)
  - West Roxbury (lat=42.27931, lon=-71.16072)
  - Roxbury (lat=42.32405, lon=-71.08522)
  - Mattapan (lat=42.27588, lon=-71.0918)
  - Dorchester (lat=42.29427, lon=-71.05185)
  - Boston (lat=42.35105, lon=-71.07638)
  - Charlestown (lat=42.37975, lon=-71.06167)
  - Winthrop (lat=42.37724, lon=-70.98115)
  - Alls

## Norfolk

In [9]:
norfolk = pd.read_csv("norfolk.csv")

mclp(norfolk, p = 2)

Selected facility locations: [(42.20386, -71.0022), (42.2366, -71.28421)]
Facility at Braintree (lat=42.20386, lon=-71.0022) covers:
  - Quincy (lat=42.23971, lon=-71.01802)
  - Braintree (lat=42.20386, lon=-71.0022)
  - Randolph (lat=42.17506, lon=-71.05011)
  - Milton (lat=42.23854, lon=-71.08464)
  - Canton (lat=42.17504, lon=-71.12645)
  - Quincy (lat=42.28709, lon=-71.02604)
  - Quincy (lat=42.26644, lon=-71.01621)
  - South Weymouth (lat=42.16625, lon=-70.95253)
  - Weymouth (lat=42.20724, lon=-70.95764)
  - East Weymouth (lat=42.2111, lon=-70.93135)
  - Holbrook (lat=42.14733, lon=-71.00484)
  - North Weymouth (lat=42.24434, lon=-70.94378)
  - Avon (lat=42.12666, lon=-71.04922)
total population covered: 301605.0

Facility at Dover (lat=42.2366, lon=-71.28421) covers:
  - Norwood (lat=42.18563, lon=-71.19464)
  - Dedham (lat=42.24665, lon=-71.1777)
  - Needham (lat=42.27615, lon=-71.24387)
  - Walpole (lat=42.14689, lon=-71.25916)
  - Wellesley Hills (lat=42.31064, lon=-71.27779)

### Worcester 

In [11]:
worcester = pd.read_csv("worcester.csv")

mclp(worcester, p = 1)

Selected facility locations: [(42.22684, -71.79037)]
Facility at Worcester (lat=42.22684, lon=-71.79037) covers:
  - Worcester (lat=42.25085, lon=-71.7683)
  - Shrewsbury (lat=42.28419, lon=-71.71558)
  - Worcester (lat=42.28979, lon=-71.78802)
  - Worcester (lat=42.24645, lon=-71.80968)
  - Worcester (lat=42.27219, lon=-71.84946)
  - Worcester (lat=42.28592, lon=-71.82853)
  - Worcester (lat=42.31367, lon=-71.79614)
  - Worcester (lat=42.24373, lon=-71.84429)
  - Auburn (lat=42.19689, lon=-71.84549)
  - Millbury (lat=42.19228, lon=-71.77397)
  - Sutton (lat=42.13387, lon=-71.75179)
  - Worcester (lat=42.22684, lon=-71.79037)
  - Grafton (lat=42.20077, lon=-71.67892)
  - North Grafton (lat=42.23189, lon=-71.69219)
  - Leicester (lat=42.24919, lon=-71.91989)
  - Worcester (lat=42.2621, lon=-71.80145)
  - South Grafton (lat=42.17496, lon=-71.67762)
  - Rochdale (lat=42.20241, lon=-71.90995)
  - Cherry Valley (lat=42.23945, lon=-71.87702)
  - North Oxford (lat=42.16177, lon=-71.89217)
tot

### Middlesex

In [12]:
middlesex = pd.read_csv("middlesex.csv")

mclp(middlesex, p = 2)

Selected facility locations: [(42.39602, -71.17955), (42.59156, -71.35553)]
Facility at Belmont (lat=42.39602, lon=-71.17955) covers:
  - Cambridge (lat=42.373778, lon=-71.106896)
  - Somerville (lat=42.39145333333334, lon=-71.10275666666666)
  - Malden (lat=42.43048, lon=-71.05762)
  - Waltham (lat=42.38714, lon=-71.23950333333333)
  - Medford (lat=42.42324, lon=-71.10878)
  - Everett (lat=42.40643, lon=-71.05446)
  - Arlington (lat=42.41825, lon=-71.165515)
  - Woburn (lat=42.48694, lon=-71.15415)
  - Watertown (lat=42.37, lon=-71.17713)
  - Lexington (lat=42.448105, lon=-71.22842)
  - Belmont (lat=42.39602, lon=-71.17955)
  - Stoneham (lat=42.47396, lon=-71.09705)
  - Winchester (lat=42.45177, lon=-71.14628)
  - Newton Center (lat=42.31515, lon=-71.19174)
  - Newton (lat=42.35294, lon=-71.18763)
  - Weston (lat=42.35893, lon=-71.30009)
  - West Newton (lat=42.34969, lon=-71.22646)
  - Newtonville (lat=42.35206, lon=-71.20835)
  - Auburndale (lat=42.34541, lon=-71.24759)
  - Newton H

### Essex

In [14]:
essex = pd.read_csv("essex.csv")

mclp(essex, p = 1)

Selected facility locations: [(42.51298, -70.90266)]
Facility at Salem (lat=42.51298, lon=-70.90266) covers:
  - Peabody (lat=42.53374, lon=-70.97222)
  - Lynn (lat=42.4712, lon=-70.93983)
  - Salem (lat=42.51298, lon=-70.90266)
  - Beverly (lat=42.56841, lon=-70.8647)
  - Saugus (lat=42.46922, lon=-71.01299)
  - Danvers (lat=42.57398, lon=-70.94923)
  - Lynn (lat=42.46355, lon=-70.9733)
  - Marblehead (lat=42.49924, lon=-70.86348)
  - Lynn (lat=42.49151, lon=-70.97166)
  - Swampscott (lat=42.47572, lon=-70.90684)
  - Wenham (lat=42.60078, lon=-70.8826)
  - Nahant (lat=42.42856, lon=-70.92614)
  - Lynn (lat=42.46125, lon=-70.94639)
  - Prides Crossing (lat=42.56294, lon=-70.82996)
  - Hathorne (lat=42.58579, lon=-70.98406)
total population covered: 341839.0

